In [1]:
# Load the weather and spot market data from the DB

import duckdb
import pandas as pd

data_start = "2023-01-01"
data_end = "2025-07-01"

db_filepath = "data/db/local.db"
con = duckdb.connect(db_filepath)
weather_cols = [
    "temperature_2m_degc",
    "shortwave_radiation_wm2",
    "direct_radiation_wm2",
    "diffuse_radiation_wm2",
    "direct_normal_irradiance_wm2",
    "global_tilted_irradiance_wm2",
    "terrestrial_radiation_wm2",
    "wind_speed_10m_kmh",
    "wind_speed_80m_kmh",
    "wind_speed_120m_kmh",
    "cloud_cover_perc",
    "cloud_cover_low_perc",
    "cloud_cover_mid_perc",
    "cloud_cover_high_perc",
    "visibility_m",
]
w_cols = ", ".join([f"open_meteo_agg_hourly.{col}" for col in weather_cols])
market_cols = [
    "non_ren_prod_kw",
    "ren_prod_kw",
    "load_kw",
    "daa_price_eurmwh",
    "idc_av_price_eurmwh",
    "idc_low_price_eurmwh",
    "idc_high_price_eurmwh",
]
m_cols = ", ".join([f"epex_market.{col}" for col in market_cols])

data = con.sql(f"""
              SELECT open_meteo_agg_hourly.ts, {w_cols}, {m_cols}
              FROM open_meteo_agg_hourly
              JOIN epex_market ON open_meteo_agg_hourly.ts = epex_market.ts
              WHERE open_meteo_agg_hourly.ts >= '{data_start}'
                AND open_meteo_agg_hourly.ts < '{data_end}'
              ORDER BY open_meteo_agg_hourly.ts
              """).df()
data.index = pd.Index(data["ts"])
data = data.drop("ts", axis=1)

In [2]:
# Preprocess the data and create sets / loaders

# Copy the original
tdata = data.copy()

# We can only use EPEX values from the previous day
last_weather_col_idx = tdata.columns.tolist().index(weather_cols[-1])
first_weather_col_idx = tdata.columns.tolist().index(weather_cols[0])
today_cols = tdata.columns[0:last_weather_col_idx]
tdata[today_cols] = tdata[today_cols].shift(-24)
tdata = tdata[:-24]

# Fill N/A values
na_columns = tdata.columns[tdata.isna().any()].tolist()
for col in na_columns:
    tdata[col] = tdata[col].fillna(tdata[col].mean())

# Fill in missing daylight-saving hours
for idx in range(1, tdata.shape[0]):
    ts = tdata.index[idx]
    prev_ts = tdata.index[idx - 1]
    diff = (ts.hour - prev_ts.hour) % 24
    if diff != 1:
        iso_str = f"{ts.year}-{ts.month:02d}-{ts.day:02d}T{(ts.hour - 1):02d}:00:00"
        new_ts = pd.to_datetime(iso_str)
        tdata = pd.concat(
            [
                tdata,
                pd.DataFrame(tdata.loc[prev_ts].to_dict(), index=[new_ts]),
            ]
        )
tdata.sort_index(ascending=True, inplace=True)

# Note down the price amplitude before normalizing
price_mean = tdata["idc_av_price_eurmwh"].mean()
price_std = tdata["idc_av_price_eurmwh"].std()

# Normalize with min/max for the fixed date columns, mean for the rest
for c in tdata.columns[1:first_weather_col_idx]:
    tdata[c] = (tdata[c] - tdata[c].min()) / (tdata[c].max() - tdata[c].min())
tdata[tdata.columns[first_weather_col_idx:]] = (
    tdata[tdata.columns[first_weather_col_idx:]]
    - tdata[tdata.columns[first_weather_col_idx:]].mean()
) / tdata[tdata.columns[first_weather_col_idx:]].std()

# Create training and test rows
split_idx = int((0.8 * len(tdata)) // 24 * 24)
iso_str = f"{tdata.index[split_idx]}"[:10]
cutoff_date = pd.to_datetime(iso_str)
train_data = tdata.loc[(tdata.index < cutoff_date)]
test_data = tdata.loc[(tdata.index >= cutoff_date)]
# train_data = train_data.drop("daa_price_eurmwh", axis=1)
# test_data = test_data.drop("daa_price_eurmwh", axis=1)
train_rows = train_data.values.astype("float32")
test_rows = test_data.values.astype("float32")
price_idx = train_data.columns.tolist().index("idc_av_price_eurmwh")

In [3]:
# Create datasets and loaders

from numpy import ndarray
import torch
from torch.utils.data import DataLoader, TensorDataset


# Datasets are created from sliding windows
def create_dataset(
    data: ndarray, lookback_hours: int, predict_hours: int
) -> tuple[torch.Tensor, torch.Tensor]:
    X, y = [], []
    for i in range(lookback_hours, len(data) - predict_hours, 24):
        feature = data[i - lookback_hours : i]
        target = data[i : i + predict_hours][:, price_idx]
        X.append(feature)
        y.append(target)
    return torch.tensor(X), torch.tensor(y)


# Create the sets
lookback_hours = 1 * 24
predict_hours = 1 * 24
X_train, y_train = create_dataset(train_rows, lookback_hours, predict_hours)
X_test, y_test = create_dataset(test_rows, lookback_hours, predict_hours)

# Loaders
train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=8, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=8, shuffle=False)

/var/folders/mt/gxps7b2d6l3dps4vtczb2rh80000gn/T/ipykernel_63082/1157537686.py:18: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:256.)
  return torch.tensor(X), torch.tensor(y)


In [4]:
# Create the base model

from torch import nn, optim


class GRUModel(nn.Module):
    def __init__(self, input_size: int, num_layers=2, hidden_size=128, dropout=0.2):
        super().__init__()
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Sequential(
            nn.Linear(hidden_size, hidden_size // 2),
            nn.Linear(hidden_size // 2, 8),
            nn.Linear(8, 1),
        )

    def forward(self, x):
        x, _ = self.gru(x)
        x = self.dropout(x)
        x = self.fc(x)
        return x


device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
# Iterate hyperparams, train and evaluate the model

import numpy as np

max_epochs = 200
patience = 10
blend_fac = 0.2

# Print the header
print(
    "# Layers\tHidden Size\t# Params\tBlend Factor\tVal. Loss\tØ Diff DAA (EUR)\tσ Diff DAA (EUR)\tØ Diff Pred. (EUR)\tσ Diff Pred. (EUR)\tØ Diff Blended (EUR)\tσ Diff Blended (EUR)"
)

for hidden_size in [64, 96, 128, 192, 256, 384]:
    for num_layers in [2, 3, 4]:
        for run in range(5):
            # Create the model
            loss_fn = nn.MSELoss()
            model = GRUModel(X_train.shape[2], num_layers, hidden_size).to(device)
            optimizer = optim.Adam(model.parameters(), lr=1e-3)
            scheduler = optim.lr_scheduler.ReduceLROnPlateau(
                optimizer, mode="min", factor=0.5, patience=5
            )
            trainable_params = sum(
                p.numel() for p in model.parameters() if p.requires_grad
            )

            # Train the model until there's no more improvement
            best_val_loss = float("inf")
            best_weights = None
            patience_counter = 0
            for epoch in range(max_epochs):
                model.train()
                train_loss = 0
                for X_batch, y_batch in train_loader:
                    X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                    optimizer.zero_grad()
                    output = model(X_batch)
                    loss = loss_fn(output, torch.unsqueeze(y_batch, 2))
                    loss.backward()
                    nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                    optimizer.step()
                    train_loss += loss.item()
                train_loss /= len(train_loader)

                # Calculate validation loss
                model.eval()
                val_loss = 0
                with torch.no_grad():
                    for X_batch, y_batch in test_loader:
                        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                        output = model(torch.squeeze(X_batch))
                        val_loss += loss_fn(output, torch.unsqueeze(y_batch, 2)).item()
                val_loss /= len(test_loader)
                scheduler.step(val_loss)

                # Stop early if there's no more improvement
                if val_loss < best_val_loss:
                    best_val_loss = val_loss
                    best_weights = model.state_dict()
                    patience_counter = 0
                else:
                    patience_counter += 1
                    if patience_counter >= patience:
                        break

            # Evaluate the price difference
            with torch.no_grad():
                model.cpu()
                # Get the predicted data
                y_pred_test = model(X_test).clone().detach().cpu().numpy()
                y_pred_test = y_pred_test.reshape(
                    y_pred_test.shape[0] * y_pred_test.shape[1]
                )
                test_plot = np.ones_like(tdata["idc_av_price_eurmwh"]) * np.nan
                test_plot[
                    len(train_rows) + lookback_hours : len(train_rows)
                    + lookback_hours
                    + y_pred_test.shape[0]
                ] = y_pred_test

                # Transform it back
                real_price = (
                    tdata["idc_av_price_eurmwh"].to_numpy() * price_std + price_mean
                )
                daa_price = (
                    tdata["daa_price_eurmwh"].to_numpy() * price_std + price_mean
                )
                pred_test_price = test_plot * price_std + price_mean

                # Calculate the blended price
                blend_price = (1 - blend_fac) * daa_price + blend_fac * pred_test_price

                # Calculate the absolute diff meand and stddev
                pdata = pd.DataFrame({"intraday_price": real_price}, index=tdata.index)
                daa_diff = pdata["intraday_price"] - daa_price
                daa_adm = daa_diff.abs().mean()
                daa_ads = daa_diff.abs().std()
                pred_diff = pdata["intraday_price"] - pred_test_price
                pred_adm = pred_diff.abs().mean()
                pred_ads = pred_diff.abs().std()
                blend_diff = pdata["intraday_price"] - blend_price
                blend_adm = blend_diff.abs().mean()
                blend_ads = blend_diff.abs().std()

                # Print the result
                print(
                    f"{num_layers}\t{hidden_size}\t{trainable_params}\t{blend_fac}\t{best_val_loss:.4f}\t{daa_adm:.3f}\t{daa_ads:.3f}\t{pred_adm:.3f}\t{pred_ads:.3f}\t{blend_adm:.3f}\t{blend_ads:.3f}"
                )

# Layers	Hidden Size	# Params	Blend Factor	Val. Loss	Ø Diff DAA (EUR)	σ Diff DAA (EUR)	Ø Diff Pred. (EUR)	σ Diff Pred. (EUR)	Ø Diff Blended (EUR)	σ Diff Blended (EUR)
2	64	44209	0.2	0.2314	12.781	33.986	20.378	20.553	11.837	13.376
2	64	44209	0.2	0.2315	12.781	33.986	20.710	19.726	12.113	13.410
2	64	44209	0.2	0.2317	12.781	33.986	20.581	20.285	11.960	13.406
2	64	44209	0.2	0.2296	12.781	33.986	20.367	20.060	11.891	13.284
2	64	44209	0.2	0.2345	12.781	33.986	21.196	21.272	12.019	13.399
3	64	69169	0.2	0.2349	12.781	33.986	20.791	21.271	11.862	13.472
3	64	69169	0.2	0.2324	12.781	33.986	20.246	20.212	11.929	13.295
3	64	69169	0.2	0.2375	12.781	33.986	20.491	20.611	11.940	13.404
3	64	69169	0.2	0.2386	12.781	33.986	21.062	20.977	11.890	13.337
3	64	69169	0.2	0.2447	12.781	33.986	21.105	20.865	12.174	13.449
4	64	94129	0.2	0.2289	12.781	33.986	20.700	20.475	12.098	13.331
4	64	94129	0.2	0.2364	12.781	33.986	20.634	20.884	11.859	13.320
4	64	94129	0.2	0.2333	12.781	33.986	20.547	20.775	11.972	13.500
4